# Content-based Filtering

In this notebook, a content-based filtering will be implemented using a neural network to build a recommender system for movies.

The code is adapted from Andrew Ng's 'Machine Learning Specialiation' course to accomplish the homework assignment. 


# Outline
- [ 1 - Import Packages](#1)
- [ 2 - Load the Data](#2)
- [ 3 - Main Functions](#3)
- [ 4 - Train the Network](#5)
- [ 5 - Visualize the results](#4)

<a name="1"></a>
## 1 - Import Packages

In [21]:
import numpy as np
import numpy.ma as ma
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from numpy import genfromtxt
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split
import tabulate
pd.set_option("display.precision", 1)
import csv
from collections import defaultdict

<a name="2"></a>
## 2 - Load the Data

In [18]:
item_train = genfromtxt('Content_data/content_item_train.csv', delimiter=',')
user_train = genfromtxt('Content_data/content_user_train.csv', delimiter=',')
y_train    = genfromtxt('Content_data/content_y_train.csv', delimiter=',')
with open('Content_data/content_item_train_header.txt', newline='') as f:    #csv reader handles quoted strings better
    item_features = list(csv.reader(f))[0]
with open('Content_data/content_user_train_header.txt', newline='') as f:
     user_features = list(csv.reader(f))[0]
item_vecs = genfromtxt('Content_data/content_item_vecs.csv', delimiter=',')

movie_dict = defaultdict(dict)
count = 0
#    with open('./data/movies.csv', newline='') as csvfile:
with open('Content_data/content_movie_list.csv', newline='') as csvfile:
    reader = csv.reader(csvfile, delimiter=',', quotechar='"')
    for line in reader:
        if count == 0:
            count += 1  #skip header
            #print(line) print
        else:
            count += 1
            movie_id = int(line[0])
            movie_dict[movie_id]["title"] = line[1]
            movie_dict[movie_id]["genres"] = line[2]


In [22]:
num_user_features = user_train.shape[1] - 3  # remove userid, rating count and ave rating during training
num_item_features = item_train.shape[1] - 1  # remove movie id at train time
uvs = 3  # user genre vector start
ivs = 3  # item genre vector start
u_s = 3  # start of columns to use in training, user
i_s = 1  # start of columns to use in training, items

In [23]:
# scale training data
item_train_unscaled = item_train
user_train_unscaled = user_train
y_train_unscaled    = y_train

scalerItem = StandardScaler()
scalerItem.fit(item_train)
item_train = scalerItem.transform(item_train)

scalerUser = StandardScaler()
scalerUser.fit(user_train)
user_train = scalerUser.transform(user_train)

scalerTarget = MinMaxScaler((-1, 1))
scalerTarget.fit(y_train.reshape(-1, 1))
y_train = scalerTarget.transform(y_train.reshape(-1, 1))
#ynorm_test = scalerTarget.transform(y_test.reshape(-1, 1))

print(np.allclose(item_train_unscaled, scalerItem.inverse_transform(item_train)))
print(np.allclose(user_train_unscaled, scalerUser.inverse_transform(user_train)))

True
True


In [24]:
# To evaluate the results, split the data into training and test sets
item_train, item_test = train_test_split(item_train, train_size=0.80, shuffle=True, random_state=1)
user_train, user_test = train_test_split(user_train, train_size=0.80, shuffle=True, random_state=1)
y_train, y_test       = train_test_split(y_train,    train_size=0.80, shuffle=True, random_state=1)
print(f"movie/item training data shape: {item_train.shape}")
print(f"movie/item test data shape: {item_test.shape}")

movie/item training data shape: (40707, 17)
movie/item test data shape: (10177, 17)


<a name="3"></a>
## 3 - Functions

## 3.1 - Main Functions

In [25]:
# construct a neural network 
num_outputs = 32
tf.random.set_seed(1)
user_NN = tf.keras.models.Sequential([   
    tf.keras.layers.Dense(units=256, activation = 'relu'),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(num_outputs)
])

item_NN = tf.keras.models.Sequential([ 
    tf.keras.layers.Dense(units=256, activation = 'relu'),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(num_outputs)
])

# create the user input and point to the base network
input_user = tf.keras.layers.Input(shape=(num_user_features))
vu = user_NN(input_user)
vu = tf.linalg.l2_normalize(vu, axis=1)

# create the item input and point to the base network
input_item = tf.keras.layers.Input(shape=(num_item_features))
vm = item_NN(input_item)
vm = tf.linalg.l2_normalize(vm, axis=1)

# compute the dot product of the two vectors vu and vm
output = tf.keras.layers.Dot(axes=1)([vu, vm])

# specify the inputs and output of the model
model = tf.keras.Model([input_user, input_item], output)

model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 14)]                 0         []                            
                                                                                                  
 input_2 (InputLayer)        [(None, 16)]                 0         []                            
                                                                                                  
 sequential (Sequential)     (None, 32)                   40864     ['input_1[0][0]']             
                                                                                                  
 sequential_1 (Sequential)   (None, 32)                   41376     ['input_2[0][0]']             
                                                                                              

## 3.2 Helper Funcitons

In [31]:
def gen_user_vecs(user_vec, num_items):
    """ given a user vector return:
        user predict maxtrix to match the size of item_vecs """
    user_vecs = np.tile(user_vec, (num_items, 1))
    return user_vecs

In [33]:
def print_pred_movies(y_p, item, movie_dict, maxcount=10):
    """ print results of prediction of a new user. inputs are expected to be in
        sorted order, unscaled. """
    count = 0
    disp = [["y_p", "movie id", "rating ave", "title", "genres"]]

    for i in range(0, y_p.shape[0]):
        if count == maxcount:
            break
        count += 1
        movie_id = item[i, 0].astype(int)
        disp.append([np.around(y_p[i, 0], 1), item[i, 0].astype(int), np.around(item[i, 2].astype(float), 1),
                     movie_dict[movie_id]['title'], movie_dict[movie_id]['genres']])

    table = tabulate.tabulate(disp, tablefmt='html', headers="firstrow")
    return table

<a name="4"></a>
## 4 - Train the Network

In [26]:
# use a mean squared error loss and an Adam optimizer.
tf.random.set_seed(1)
cost_fn = tf.keras.losses.MeanSquaredError()
opt = keras.optimizers.Adam(learning_rate=0.01)
model.compile(optimizer=opt,
              loss=cost_fn)

tf.random.set_seed(1)
model.fit([user_train[:, u_s:], item_train[:, i_s:]], y_train, epochs=30)

Epoch 1/30
1273/1273 [==============================] - 3s 1ms/step - loss: 0.1235
Epoch 2/30
1273/1273 [==============================] - 2s 1ms/step - loss: 0.1139
Epoch 3/30
1273/1273 [==============================] - 2s 1ms/step - loss: 0.1089
Epoch 4/30
1273/1273 [==============================] - 2s 1ms/step - loss: 0.1048
Epoch 5/30
1273/1273 [==============================] - 2s 1ms/step - loss: 0.1023
Epoch 6/30
1273/1273 [==============================] - 2s 1ms/step - loss: 0.0994
Epoch 7/30
1273/1273 [==============================] - 2s 1ms/step - loss: 0.0971
Epoch 8/30
1273/1273 [==============================] - 2s 1ms/step - loss: 0.0949
Epoch 9/30
1273/1273 [==============================] - 2s 1ms/step - loss: 0.0932
Epoch 10/30
1273/1273 [==============================] - 2s 1ms/step - loss: 0.0917
Epoch 11/30
1273/1273 [==============================] - 2s 1ms/step - loss: 0.0902
Epoch 12/30
1273/1273 [==============================] - 2s 1ms/step - loss: 0.0884
E

In [27]:
model.evaluate([user_test[:, u_s:], item_test[:, i_s:]], y_test)

319/319 [==============================] - 0s 842us/step - loss: 0.0818


0.08178229629993439

It is comparable to the training loss indicating the model has not substantially overfit the training data.

<a name="5"></a>
## 5 - Visualize the results

## 5.1 - Predictions for a new user

In [28]:
new_user_id = 5000
new_rating_ave = 0.0
new_action = 0.0
new_adventure = 5.0
new_animation = 0.0
new_childrens = 0.0
new_comedy = 0.0
new_crime = 0.0
new_documentary = 0.0
new_drama = 0.0
new_fantasy = 5.0
new_horror = 0.0
new_mystery = 0.0
new_romance = 0.0
new_scifi = 0.0
new_thriller = 0.0
new_rating_count = 3

user_vec = np.array([[new_user_id, new_rating_count, new_rating_ave,
                      new_action, new_adventure, new_animation, new_childrens,
                      new_comedy, new_crime, new_documentary,
                      new_drama, new_fantasy, new_horror, new_mystery,
                      new_romance, new_scifi, new_thriller]])

In [34]:
# generate and replicate the user vector to match the number movies in the data set.
user_vecs = gen_user_vecs(user_vec,len(item_vecs))

# scale our user and item vectors
suser_vecs = scalerUser.transform(user_vecs)
sitem_vecs = scalerItem.transform(item_vecs)

# make a prediction
y_p = model.predict([suser_vecs[:, u_s:], sitem_vecs[:, i_s:]])

# unscale y prediction 
y_pu = scalerTarget.inverse_transform(y_p)

# sort the results, highest prediction first
sorted_index = np.argsort(-y_pu,axis=0).reshape(-1).tolist()  #negate to get largest rating first
sorted_ypu   = y_pu[sorted_index]
sorted_items = item_vecs[sorted_index]  #using unscaled vectors for display

print_pred_movies(sorted_ypu, sorted_items, movie_dict, maxcount = 10)

27/27 [==============================] - 0s 1ms/step


y_p,movie id,rating ave,title,genres
4.5,98809,3.8,"Hobbit: An Unexpected Journey, The (2012)",Adventure|Fantasy
4.4,8368,3.9,Harry Potter and the Prisoner of Azkaban (2004),Adventure|Fantasy
4.4,5816,3.6,Harry Potter and the Chamber of Secrets (2002),Adventure|Fantasy
4.4,106489,3.6,"Hobbit: The Desolation of Smaug, The (2013)",Adventure|Fantasy
4.4,81834,4,Harry Potter and the Deathly Hallows: Part 1 (2010),Action|Adventure|Fantasy
4.4,118696,3.4,The Hobbit: The Battle of the Five Armies (2014),Adventure|Fantasy
4.3,40815,3.8,Harry Potter and the Goblet of Fire (2005),Adventure|Fantasy|Thriller
4.3,54001,3.9,Harry Potter and the Order of the Phoenix (2007),Adventure|Drama|Fantasy
4.3,5952,4,"Lord of the Rings: The Two Towers, The (2002)",Adventure|Fantasy
4.3,57640,3.8,Hellboy II: The Golden Army (2008),Action|Adventure|Fantasy|Sci-Fi
